### Estimate the pifactor from HRRR
InSAR-PWV = ZWD x Pi_factor

reference: Askne and Nordius, 1987: Estimation of tropospheric delay for microwaves from surface weather data 

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cfgrib
import cf2cdm
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
from datetime import datetime, timedelta
import urllib.request
from cfgrib.xarray_store import open_dataset
import warnings
import h5py
import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy.interpolate import interp2d
from scipy.interpolate import griddata
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression

In [2]:
warnings.filterwarnings('ignore')

In [3]:
os.chdir('/data2/willytsai/InSAR_HRRR/CentralNevadaSenAT166/mintpy/pic/')
file = np.loadtxt('rms_timeseriesResidual_ramp.txt',skiprows=4)
date_acqui = []
for t in range(file.shape[0]):
    date_acqui.append(str(file[t,0])[:8]) # times of acquisitions

In [4]:
date_xr = []
for date in date_acqui:
    date_xr.append(datetime.strptime(date,'%Y%m%d') + timedelta(hours=2))

In [5]:
# get geolocation from InSAR
geo_file = '/data2/willytsai/InSAR_HRRR/CentralNevadaSenAT166/mintpy/inputs/geometryRadar.h5'
geo = h5py.File(geo_file,'r')
# for key in geo.keys():
#     print(key) #Names of the groups in HDF5 file.
lat = geo['latitude'];
lon = geo['longitude'];
incidence = geo['incidenceAngle'];
axis_bound = [np.min(lat),np.max(lat),np.min(lon),np.max(lon)]; # coordinate bound [South,North,West,East]
axis_bound = [np.unique(lat.value)[1],np.unique(lat.value)[-1],np.unique(lon.value)[0],np.unique(lon.value)[-2]]
axis_bound

[37.228, 43.223568, -119.00549, -114.65358]

In [6]:
data_ifgrams = h5py.File('/data2/willytsai/InSAR_HRRR/CentralNevadaSenAT166/mintpy/S1_IW123_166_0121_0140_20150322_XXXXXXXX.he5', 'r')
geo_info = data_ifgrams['HDFEOS']['GRIDS']['timeseries']['geometry']
lon_ifgrams = geo_info['longitude']
lat_ifgrams = geo_info['latitude']

### calculate pi_factor from HRRR at acquisitions and output as one netcdf file

In [7]:
os.chdir('/data2/willytsai/InSAR_HRRR/HRRR_data/google_archive/regrid_3km/')
files = sorted(glob('*grib2'))

In [8]:
# mean temperature of water vapor Tm
tmp = xr.open_dataset('hrrr.20181026.t02z.regrid3km.grib2',engine='cfgrib',
                            backend_kwargs=dict(filter_by_keys={'typeOfLevel':'isobaricInhPa'}))
tmp = tmp.sel(latitude=slice(axis_bound[0],axis_bound[1])
             ,longitude=slice(axis_bound[2]+360,axis_bound[3]+360))

In [9]:
Ta = tmp.t # air temperature
q = tmp.q # specific humidity
rh = tmp.r
prs = Ta.isobaricInhPa*100 # Pa

In [10]:
# interpolating T an q into InSAR grids, 
def remap_hrrr2SAR(hrrr_2dmap):
    "regrid hrrr data into SAR"
    lon_hrrr,lat_hrrr = np.meshgrid(hrrr_2dmap.longitude.values,hrrr_2dmap.latitude.values)
    lon_isar,lat_isar = np.meshgrid(lon_sar,lat_sar)
    points = np.stack([lon_hrrr.ravel(),lat_hrrr.ravel()]).swapaxes(0,1)
    var_remap = griddata(points, hrrr_2dmap.values.ravel(), (lon_isar, lat_isar), method='linear')

    return var_remap

In [11]:
lon_sar = np.linspace(axis_bound[2],axis_bound[3],lon_ifgrams.shape[1])
lat_sar = np.linspace(axis_bound[0],axis_bound[1],lon_ifgrams.shape[0])

In [12]:
def Tm_calcualtion(Ta,rh,prs):

    a = 21.8745584; b = 7.66
    es = 6.1078*np.exp(a*(Ta-273.15)/(Ta-b))*100 # Pa
    e = es*rh

    Rd = 287.15; Rw = 461.52
    ro = prs/Rd/Ta

    Tm = np.trapz(e/Ta/ro,prs,axis=0)/np.trapz(e/Ta**2/ro,prs,axis=0)
    Tm = xr.DataArray(Tm,dims=('latitude','longitude')
                                ,coords=(Ta.latitude,Ta.longitude-360),name='Tm')
    return Tm

In [13]:
lon_sar.shape

(1742,)

In [14]:
%%time
Tm_all = np.zeros((len(files),len(lat_sar),len(lon_sar)))

for i,file in enumerate(files):
    
    data = xr.open_dataset(file,engine='cfgrib',
                            backend_kwargs=dict(filter_by_keys={'typeOfLevel':'isobaricInhPa'}))
    tmp = data.sel(latitude=slice(axis_bound[0],axis_bound[1])
             ,longitude=slice(axis_bound[2]+360,axis_bound[3]+360))
    Ta = tmp.t
    rh = tmp.r
    prs = Ta.isobaricInhPa*100
    Tm = Tm_calcualtion(Ta,rh,prs)
    Tm_insar = remap_hrrr2SAR(Tm)
    
    Tm_all[i,:,:] = Tm_insar
    print(file)

hrrr.20150322.t02z.regrid3km.grib2
hrrr.20150403.t02z.regrid3km.grib2
hrrr.20150415.t02z.regrid3km.grib2
hrrr.20150509.t02z.regrid3km.grib2
hrrr.20150626.t02z.regrid3km.grib2
hrrr.20150720.t02z.regrid3km.grib2
hrrr.20150906.t02z.regrid3km.grib2
hrrr.20160104.t02z.regrid3km.grib2
hrrr.20160221.t02z.regrid3km.grib2
hrrr.20160316.t02z.regrid3km.grib2
hrrr.20160409.t02z.regrid3km.grib2
hrrr.20160503.t02z.regrid3km.grib2
hrrr.20160527.t02z.regrid3km.grib2
hrrr.20160714.t02z.regrid3km.grib2
hrrr.20160807.t02z.regrid3km.grib2
hrrr.20161105.t02z.regrid3km.grib2
hrrr.20161129.t02z.regrid3km.grib2
hrrr.20161223.t02z.regrid3km.grib2
hrrr.20170116.t02z.regrid3km.grib2
hrrr.20170305.t02z.regrid3km.grib2
hrrr.20170317.t02z.regrid3km.grib2
hrrr.20170422.t02z.regrid3km.grib2
hrrr.20170516.t02z.regrid3km.grib2
hrrr.20170528.t02z.regrid3km.grib2
hrrr.20170609.t02z.regrid3km.grib2
hrrr.20170621.t02z.regrid3km.grib2
hrrr.20170703.t02z.regrid3km.grib2
hrrr.20170715.t02z.regrid3km.grib2
hrrr.20170727.t02z.r

In [21]:
# Pi_factor = 1e-5*(k2'+k3/Tm)Rw
k2_prunt = 16.52 # K/mbar
k3 = 3.776*1e5 # K^2/mbar
Rw = 461.52 
Pi_factor = 1e-5*(k2_prunt + k3/Tm_all)*Rw

In [22]:
Pi_factor.shape

(118, 2399, 1742)

In [23]:
Pi_factor_xr = xr.DataArray(Pi_factor,dims=('time','latitude','longitude'),
                           coords=(date_xr, lat_sar, lon_sar), name='Pi_factor')
Pi_factor_ds = Pi_factor_xr.to_dataset(name='Pi_factor')
Pi_factor_ds.to_netcdf('/data2/willytsai/InSAR_HRRR/HRRR_Pi_factor_NEVADA.nc')

In [24]:
data_out = xr.open_dataset('/data2/willytsai/InSAR_HRRR/HRRR_Pi_factor_NEVADA.nc')
data_out

<xarray.Dataset>
Dimensions:    (latitude: 2399, longitude: 1742, time: 118)
Coordinates:
  * time       (time) datetime64[ns] 2015-03-22T02:00:00 ... 2020-10-03T02:00:00
  * latitude   (latitude) float64 37.23 37.23 37.23 37.24 ... 43.22 43.22 43.22
  * longitude  (longitude) float64 -119.0 -119.0 -119.0 ... -114.7 -114.7 -114.7
Data variables:
    Pi_factor  (time, latitude, longitude) float64 ...